https://towardsdatascience.com/step-by-step-twitter-sentiment-analysis-in-python-d6f650ade58d

In [1]:
# Install Libraries
!pip install textblob
!pip install tweepy
!pip install pycountry
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 4.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=44e916b5ca275a9cb6db97ea4fbb5b1f8170868808215147f26d6a38c0d65238
  Stored in directory: /root/.cache/pip/wheels/e2/aa/0f/c224e473b464387170b83ca7c66947b4a7e33e8d903a679748
Successfully built pycountry
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 7.1 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-n

In [2]:
# Import Libraries
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from sklearn.feature_extraction.text import CountVectorizer
from google.colab import drive

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [3]:
# Mount GDrive
drive.mount('/gdrive')


Mounted at /gdrive


In [4]:
os.chdir('/gdrive/MyDrive/Docencia/UV-ML')
!ls

 Clientes_test.csv
 Clientes_train.csv
 df_export.xlsx
'Ejemplo Análisis de Sentimiento (Twitter).ipynb'
'Ejemplo Análisis de Sentimiento v2(Twitter).ipynb'
'Ejemplo Clasificación ML.ipynb'
'Ejemplo CNN.ipynb'
'Ejemplo CNN v2'
'Ejemplo Kmeans_con_pca.ipynb'
'Ejemplo Red Neuronal.ipynb'
'Ejemplo Regresion.ipynb'
 Zonas.csv


In [6]:
# Authentication
consumerKey = "RsXyfMlGtuc8fvRawpntG0AWf"
consumerSecret = "853aqkKZtWGS3ftbF37TYBEAe0EihNIWORTQXIwF5rzrhooDp1"
accessToken = "124614074-ZsyiKRjHBBfznU8k2nYa8H9fFUSjmROS8rEpw5J1"
accessTokenSecret = "LucPICTTmzimUd3NNWqRoNPWROQrmFWi8OjHxK04moVWC"
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [10]:
#Sentiment Analysis
def percentage(part,whole):
 return 100 * float(part)/float(whole)
# parámetros query: https://medium.com/@robguilarr/making-queries-to-twitter-api-on-tweepy-66afeb7184a4
# https://plainenglish.io/blog/twitter-sentiment-analysis-using-vader-tweepy-b2a62fba151e
keyword = '"Universidad de Valencia" OR "Universitat de València"'
#keyword = '"@SEFutbol" OR "@LUISENRIQUE21"'
#keyword = '@sanchezcastejon'
#keyword = '@Spain_AI_'
#keyword = "ropa_ de_segunda_mano OR ropa_casi_nueva"
#keyword = "@DIA_Esp"

noOfTweet = int(1000)

#searchCountry = "Spain"
#places = api.geo_search(query=searchCountry, granularity="country")
#place_id = places[0].id

tweets = tweepy.Cursor(api.search, q=(keyword),lang='es' , wait_on_rate_limit=True  ).items(noOfTweet)
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []
date_list = []
score_list = []
user_list =[]
location_list =[]
sent_list = []


In [11]:
for tweet in tweets:
 
  #print(tweet.text)
  tweet_list.append(tweet.text)
  analysis = TextBlob(tweet.text)
  score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
  date_list.append(tweet.created_at)
  score_list.append(score)
  user_list.append(tweet.user.name)
  location_list.append(tweet.user.location)

  neg = score['neg']
  neu = score['neu']
  pos = score['pos']
  comp = score['compound']
  polarity += analysis.sentiment.polarity
  
  if comp <=-0.05:
    sent_list.append("negative")
    negative_list.append(tweet.text)
    negative += 1
  elif comp >=0.05:
    sent_list.append("postive")
    positive_list.append(tweet.text)
    positive += 1
  
  else:
    sent_list.append("neutral")
    neutral_list.append(tweet.text)
    neutral += 1
positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

In [12]:
#Number of Tweets (Total, Positive, Negative, Neutral)
tweet_list_2 = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print("total number: ",len(tweet_list_2))
print("positive number: ",len(positive_list))
print("negative number: ", len(negative_list))
print("neutral number: ",len(neutral_list))

total number:  1000
positive number:  91
negative number:  185
neutral number:  724


In [ ]:
# tweet_list.to_csv("tweet_list.csv", index=False, sep=";")

In [13]:
df_export = pd.DataFrame({"Tweet":tweet_list, "Score":score_list, "Sentiment": sent_list ,"Date": date_list, "User": user_list, "Location": location_list})

In [14]:
#df_export.to_csv("df_export.csv", index=False, sep=";", encoding="utf-8")
df_export.to_excel("df_export.xlsx")

In [15]:
df_export

,Tweet,Score,Sentiment,Date,User,Location
0,@DIA_Esp #PatineteHolaCola Lo bien que me v...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral,2022-12-14 12:29:26,juanillo999 🔻,Campo de estrellas -Compostela
1,RT @DIA_Esp: ✨ ESPECIAL 100K SEGUIDORES ✨\n\n😍...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral,2022-12-14 12:20:00,Sólvy 🌃,she/her
2,RT @DIA_Esp: ⚽ No pudimos seguir en este Mundi...,"{'neg': 0.106, 'neu': 0.894, 'pos': 0.0, 'comp...",negative,2022-12-14 12:18:10,Nofuiyo 📈🌕📉🌍💥💸💀🫣,En la Luna
3,@DIA_Esp #Dia100K\nYa está. Este es el meme\n...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral,2022-12-14 12:16:51,MrGG,
4,RT @DIA_Esp: Tenemos casi 300 memardos. Pero s...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral,2022-12-14 12:16:10,Almu_81,
...,...,...,...,...,...,...
995,RT @DIA_Esp: ⚽ No pudimos seguir en este Mundi...,"{'neg': 0.106, 'neu': 0.894, 'pos': 0.0, 'comp...",negative,2022-12-11 23:44:24,Raki,He/Him
996,RT @DIA_Esp: ⚽ No pudimos seguir en este Mundi...,"{'neg': 0.106, 'neu': 0.894, 'pos': 0.0, 'comp...",negative,2022-12-11 23:33:48,༺ 🌏 𝓜𝓪𝔂𝓽𝓮 🌺༻,
997,RT @DIA_Esp: 🛴SORTEO #HOLACOLACUP🛴\n\nGana un ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral,2022-12-11 23:04:43,Sergio Gonzalez ,Neptuno
998,RT @DIA_Esp: ⚽ No pudimos seguir en este Mundi...,"{'neg': 0.106, 'neu': 0.894, 'pos': 0.0, 'comp...",negative,2022-12-11 22:50:51,DavidOria,


In [ ]:
tweet


Status(_api=<tweepy.api.API object at 0x7f123ccb9fd0>, _json={'created_at': 'Wed Dec 14 10:26:30 +0000 2022', 'id': 1602973297641414656, 'id_str': '1602973297641414656', 'text': '@nacho_aledo @SEFutbol @rfef Dice, soy Luis de la Fuente, uniformado con @SEFutbol siendo seleccionador de la Sub 2… https://t.co/2GUEbwoGOP', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'nacho_aledo', 'name': 'Atletictrainer Castellón', 'id': 1344140292, 'id_str': '1344140292', 'indices': [0, 12]}, {'screen_name': 'SEFutbol', 'name': 'Selección Española de Fútbol', 'id': 1139001474, 'id_str': '1139001474', 'indices': [13, 22]}, {'screen_name': 'rfef', 'name': 'RFEF', 'id': 137415452, 'id_str': '137415452', 'indices': [23, 28]}, {'screen_name': 'SEFutbol', 'name': 'Selección Española de Fútbol', 'id': 1139001474, 'id_str': '1139001474', 'indices': [73, 82]}], 'urls': [{'url': 'https://t.co/2GUEbwoGOP', 'expanded_url': 'https://twitter.com/i/web/status/160297